In [1]:
import pymeshlab
import numpy as np

In [2]:
ms = pymeshlab.MeshSet()

ms.load_new_mesh("model_normalized.obj")

m = ms.current_mesh()

v_matrix = m.vertex_matrix()
f_matrix = m.face_matrix()

By default faces are defined and stored in the counter-clockwise order, so should not have any issues calculating the moments

In [3]:
num_faces = len(f_matrix)
    
# Extract coordinates
A = np.zeros([num_faces,3])
B = np.zeros([num_faces,3])
C = np.zeros([num_faces,3])
for face_num in range(num_faces):
    [A[face_num,:],B[face_num,:],C[face_num,:]] = v_matrix[f_matrix[face_num]]

In [4]:
def M_ijk_fast(f_matrix,v_matrix,max_m):
    num_faces = len(f_matrix)
    
    # Extract coordinates
    A = np.zeros([num_faces,3])
    B = np.zeros([num_faces,3])
    C = np.zeros([num_faces,3])
    for face_num in range(num_faces):
        [A[face_num,:],B[face_num,:],C[face_num,:]] = v_matrix[f_matrix[face_num]]
    # Calculate Determinants
    dets = np.linalg.det([[A[i],B[i],C[i]] for i in range(len(f_matrix))])
    
    # Allocate Tensors
    M_tensor = np.zeros([num_faces,max_m,max_m,max_m])
    C_tensor = np.zeros([num_faces,max_m,max_m,max_m])
    D_tensor = np.zeros([num_faces,max_m,max_m,max_m])
    S_tensor = np.zeros([num_faces,max_m,max_m,max_m])
    
    # Calculate C Tensor, parallellized over faces
    for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        C_tensor[:,i,j,k] = (C[:,0]**i)*(C[:,1]**j)*(C[:,2]**k)*(np.math.factorial(i+j+k)/(np.math.factorial(i)*np.math.factorial(j)*np.math.factorial(k)))
                        
    # Calculate D Tensor, parallellized over faces
    for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        if (i<0) or (j<0) or (k<0):
                            # D_ijk=0
                            pass
                        elif (0==i) and (0==j) and (0==k):
                            # D_ijk = 1
                            D_tensor[:,i,j,k] = 1
                        else:
                            D_tensor[:,i,j,k] = B[:,0]*D_tensor[:,i-1,j,k]+B[:,1]*D_tensor[:,i,j-1,k]+B[:,2]*D_tensor[:,i,j,k-1]+C_tensor[:,i,j,k]
    
    
    # Calculate S Tensor, parallellized over faces
    for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        if (i<0) or (j<0) or (k<0):
                            # S_ijk = 0
                            pass
                        elif (0==i) and (0==j) and (0==k):
                            # S_ijk = 1
                            S_tensor[:,i,j,k] = 1
                        else:
                            S_tensor[:,i,j,k] = A[:,0]*S_tensor[:,i-1,j,k]+A[:,1]*S_tensor[:,i,j-1,k]+A[:,2]*S_tensor[:,i,j,k-1]+D_tensor[:,i,j,k]
        
    # Calculate M Tensor, parallellized over faces
    for i in range(max_m):
        for j in range(max_m):
            for k in range(max_m):
                if (i+j+k)<=max_m:
                    M_tensor[:,i,j,k] = ((np.math.factorial(i)*np.math.factorial(j)*np.math.factorial(k))/np.math.factorial(i+j+k+3))*dets[:]*S_tensor[:,i,j,k]
    return np.sum(M_tensor,axis=0)
#      M_tensor = M_tensor
#     return M_tensor

                        
    
    

This is the final loss function that gives the scalar.

In [5]:
def mesh_moment_loss(f_matrix_1, v_matrix_1, f_matrix_2, v_matrix_2, max_m = 5):
    return np.linalg.norm(M_ijk_fast(f_matrix_1, v_matrix_1,max_m)-M_ijk_fast(f_matrix_2, v_matrix_2,max_m))

The moments, and correspondingly the moment loss, can be made invariant to translation and scaling.
This can easily done by shifting and scaling the coordinates.

THIS IS NOT WORKING PROPERLY RIGHT NOW!
The issue seems to be with the pymeshlab library, which is supposed to load the mesh with the face vertices ordered CCW when viewed from the outside of the object.
Find another library that follows this rule.

In [6]:
# Make it invariant to translation and scaling

def mesh_moment_invariant(f_matrix_1, v_matrix_1, max_m = 5):
    
    # Obtain the scale and centroids of the object
    Tensor_1 = M_ijk_fast(f_matrix_1, v_matrix_1,2)
    print(Tensor_1)
    
    v_matrix_1 = np.array(v_matrix_1)
    
    v_matrix_1 = v_matrix_1 / np.cbrt(Tensor_1[0,0,0])
#     v_matrix_1[:,0] = v_matrix_1[:,0] - Tensor_1[1,0,0]/Tensor_1[0,0,0] 
#     v_matrix_1[:,1] = v_matrix_1[:,1] - Tensor_1[0,1,0]/Tensor_1[0,0,0] 
#     v_matrix_1[:,2] = v_matrix_1[:,2] - Tensor_1[0,0,1]/Tensor_1[0,0,0] 
    
    # Obtain the scale and centroids of the object again for better numerical precision
    # Doing this recursively keeps improving precision of the scale and centroids
    Tensor_2 = M_ijk_fast(f_matrix_1, v_matrix_1,2)
    print(Tensor_2)
    
    v_matrix_1 = v_matrix_1 / np.cbrt(Tensor_1[0,0,0])
#     v_matrix_1[:,0] = v_matrix_1[:,0] - Tensor_1[1,0,0]/Tensor_1[0,0,0] 
#     v_matrix_1[:,1] = v_matrix_1[:,1] - Tensor_1[0,1,0]/Tensor_1[0,0,0] 
#     v_matrix_1[:,2] = v_matrix_1[:,2] - Tensor_1[0,0,1]/Tensor_1[0,0,0] 
    

    M_1 = M_ijk_fast(f_matrix_1, v_matrix_1,max_m)
    
#     for i in range(max_m):
#             for j in range(max_m):
#                 for k in range(max_m):
#                     if (i+j+k)<=max_m:
#                         M_1[i,j,k] = M_1[i,j,k] / np.power(Tensor_1[0,0,0],(i+j+k+3)/3)
    
    return M_1

Note that this loss function is directly comparing the moments.

Matching the moment series will make two shapes match each other.

However, if we want to make the loss function invariant to scale, translation, and rotation, (or any symmetries) then the invariants of the loss function can be calcuated from the moments tensor we have just calculated.
Matching the invariants will make two shapes "match" each other without being affected by scale, translation, and rotation.

This is easy to implement, will do this next.

In [7]:
mesh_moment_invariant(f_matrix,v_matrix,max_m=3)

[[[-1.98788045e-20  2.92615142e-19]
  [ 7.66589633e-21  9.39987369e-21]]

 [[ 2.54016826e-21 -5.20477551e-23]
  [-7.89763615e-23  0.00000000e+00]]]
[[[-1.58700684e+02  6.42263040e+07]
  [-5.75091200e+06  4.02307573e+13]]

 [[-1.16475200e+06 -1.31755672e+12]
  [ 8.73254093e+10  0.00000000e+00]]]


array([[[-2.73447761e+21, -4.20463462e+34, -1.18258674e+47],
        [ 4.92628830e+33,  1.23459016e+46,  4.99876819e+58],
        [-3.67443978e+45, -1.09986726e+58,  0.00000000e+00]],

       [[-2.39860786e+32, -6.11207021e+44, -6.04238448e+57],
        [ 8.36931287e+43,  7.95299336e+56,  0.00000000e+00],
        [-1.52787112e+56,  0.00000000e+00,  0.00000000e+00]],

       [[-5.48777457e+44, -4.56328982e+56,  0.00000000e+00],
        [-1.33840438e+56,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00]]])

In [8]:
%timeit M_ijk_fast(f_matrix,v_matrix,10)

215 ms ± 2.33 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
